In [2]:

# import necessary modules
#%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from numpy.fft import fft, ifft , rfft, irfft , fftfreq
from numpy import exp, log, log10, cos, sin, pi, cosh, sinh , sqrt
from classy import Class
from scipy.optimize import fsolve
from scipy.special import gamma
from scipy.special import hyp2f1
from scipy import interpolate
import sys
from time import time
from scipy.integrate import quad
import scipy.integrate as integrate
from scipy import special
#from scipy.special import factorial
from scipy.misc import factorial # EM: use deprecated .misc version of factorial
import math

In [3]:
############################################
z_pk = 0.61
common_settings = {# fixed LambdaCDM parameters
#                    'N_eff':3.046,
                   'N_ncdm':1, 
                   'N_ur':2.0328, 
                   'm_ncdm': 0.06, 
                   'ln10^{10}A_s':3.044,
                   'n_s':0.9649,
                   'tau_reio':0.052,
                   'omega_b':0.02237,
                   'omega_cdm':0.12,
                   'h':0.6737,
                   'YHe':0.25,
                   # other output and precision parameters
                   'P_k_max_1/Mpc':100.0,
                   'z_pk':z_pk}  

EDE_common_settings = { #fixed EDE parameters
         'fEDE': 0.122,
         'log10z_c': 3.562,
#          'f_scf':'3.98e+26',
#          'm_scf':'5.31e-28',
         'thetai_scf': 2.83,
         'h': .7219,
#          '100*theta_s': 1.04152,
         'A_s': 2.215e-09,
         'n_s': 0.9889,
         'omega_b': 0.02253,
         'omega_cdm': 0.1306,
         'tau_reio': 0.072,
#      'N_eff':3.046,
#     'N_ncdm':0,
         'N_ncdm':1, 
         'N_ur':2.0328, 
          'm_ncdm': 0.06, 
         'YHe':0.25,
         'Omega_Lambda':0.0, 
         'Omega_fld':0, 
         'Omega_scf':-1, 
         'n_scf':3, 
         'CC_scf':1, 
         'scf_parameters':'1, 1, 1, 1, 1, 0.0', 
         'scf_tuning_index':3, 
         'attractor_ic_scf':'no', 
         'z_pk':z_pk}

M = Class()
EDE= Class()

M.set(common_settings)
EDE.set(EDE_common_settings)
#let's first take a look at the one-loop power spectrum for matter without IR resummation
M.set({'output':'mPk',
       'non linear':'PT',
       'IR resummation':'No',
       'Bias tracers':'No'})

EDE.set({'output':'mPk',
       'non linear':'PT',
       'IR resummation':'No',
       'Bias tracers':'No'})

M.compute()

EDE.compute()
# EM: this takes an insanely long time!

NameError: name 'EDE1' is not defined

In [6]:
#now we compute all the spectra including IR resummation, RSD, 
#and AP generated for a fiducial cosmology with $\Omega_m=0.31$ 
M1 = Class()
M1.set(common_settings)
M1.set({'output':'mPk,tCl',
        'non linear':'PT',
        'IR resummation':'Yes',
        'Bias tracers':'Yes',
        'RSD':'Yes',
        'AP':'Yes',
        'Omfid':'0.31'
       })
M1.compute()

In [10]:
EDE1 = Class()
EDE1.set(EDE_common_settings)
EDE1.set({'output':'mPk,tCl',
        'non linear':'PT',
        'IR resummation':'Yes',
        'Bias tracers':'Yes',
        'RSD':'Yes',
        'AP':'Yes',
        'Omfid':'0.31' #EM left Omegafid fixed to LCDM value. MI Should not be changed, value used by BOSS! 
       })
EDE1.compute()

In [11]:
# esthetic definitions for the plots
font = {'size'   : 10, 'family':'STIXGeneral'}
axislabelfontsize='large'
matplotlib.rc('font', **font)
matplotlib.mathtext.rcParams['legend.fontsize']='small'
plt.rcParams["figure.figsize"] = [10.0,8.0]

In [12]:
print EDE1.Omega_m()
print EDE1.sigma8()
print EDE1.h()
print M1.Omega_m()
print M1.sigma8()




0.295072267204
0.849864189119
0.7219
0.315098163322
0.810752940718


In [13]:
#############################################
#
# extract LCDM spectra 
#
#############################################

h = M.h()
fz = M.scale_independent_growth_factor_f(z_pk)
kvec = np.logspace(-3,np.log10(3),1000) # array of kvec in h/Mpc
twopi = 2.*math.pi
khvec = kvec*h # array of kvec in 1/Mpc


legarray = []
pk_lin = []
pk_loop = []
pk_tree = []
pk_full = []
pk_full_ir = []
pk_ctr = []
pk_gg = []
pk_gm = []
pk_m0 = []
pk_m2 = []
pk_m4 = []
pk_g0 = []
pk_g2 = []
pk_g4 = []
pk_Id2 = []
pk_IG2 = []
pk_Id2d2 = []
pk_Id2G2 = []
pk_IG2G2 = []
pk_FG2 = []

b1 = 1.819736572651593
cs = 1. # in units [Mpc/h]^2
b2 = -3.5142879639974764
bG2 = 0.6942619836144907
bGamma3 = 0.
Pshot = 7431.955177445927 # in units [Mpc/h]^3
cs0 = -10.711564425954903 # in units [Mpc/h]^2
cs2 = -46.19171787820874 # in units [Mpc/h]^2
cs4 = -5. # in units [Mpc/h]^2
b4 = 266.68840332551514 # in units [Mpc/h]^4

for kh in khvec:
    pk_tree.append(M.pk(kh,z_pk)[14]*h**3.)
    pk_loop.append(M.pk(kh,z_pk)[0]*h**3.)
    pk_ctr.append(2*M.pk(kh,z_pk)[10]*h)
    # basic real space matter power spectrum without IR resummation below
    pk_full.append((M.pk(kh,z_pk)[0]+M.pk(kh,z_pk)[14]+2*cs*M.pk(kh,z_pk)[10]/h**2.)*h**3.)
    # linear theory matter power spectrum below
    pk_lin.append(M1.pk_lin(kh,z_pk)*h**3.)
    # real space matter power spectrum below
    pk_full_ir.append((M1.pk(kh,z_pk)[0]+M1.pk(kh,z_pk)[14]+2*cs*M1.pk(kh,z_pk)[10]/h**2.)*h**3.)
    # real space galaxy-galaxy power spectrum below
    pk_gg.append((b1**2.*M1.pk(kh, z_pk)[14] 
                  + b1**2.*M1.pk(kh, z_pk)[0] 
                  + 2.*cs0*M1.pk(kh, z_pk)[10]/h**2. 
                  + b1*b2*M1.pk(kh, z_pk)[2]
                  + 0.25*b2**2.*M1.pk(kh, z_pk)[1] 
                  + 2.*b1*bG2*M1.pk(kh, z_pk)[3] 
                  + b1*(2.*bG2 + 0.8*bGamma3)*M1.pk(kh, z_pk)[6] 
                  + bG2**2.*M1.pk(kh, z_pk)[5] 
                  + b2*bG2*M1.pk(kh, z_pk)[4])*h**3. + Pshot) 
    # real space galaxy-matter power spectrum below
    pk_gm.append((b1*M1.pk(kh, z_pk)[14] 
                  + b1*M1.pk(kh, z_pk)[0] 
                  + 2.*cs0*M1.pk(kh, z_pk)[10]/h**2. 
                  + (b2/2)*M1.pk(kh, z_pk)[2]
                  + bG2*M1.pk(kh, z_pk)[3])*h**3.) 
    # separate contributions into the galaxy-galaxy power spectrum
    pk_Id2.append((M1.pk(kh, z_pk)[2])*h**3.) 
    pk_IG2.append((M1.pk(kh, z_pk)[3])*h**3.) 
    pk_Id2d2.append((M1.pk(kh, z_pk)[1])*h**3.) 
    pk_IG2G2.append((M1.pk(kh, z_pk)[5])*h**3.) 
    pk_Id2G2.append((M1.pk(kh, z_pk)[4])*h**3.) 
    pk_FG2.append((M1.pk(kh, z_pk)[6])*h**3.) 
    # dark matter redsfhit space monopole below
    pk_m0.append((M1.pk(kh, z_pk)[15] 
                  +M1.pk(kh, z_pk)[21]
                  +M1.pk(kh, z_pk)[16] 
                  +M1.pk(kh, z_pk)[22] 
                  +M1.pk(kh, z_pk)[17] 
                  +M1.pk(kh, z_pk)[23] 
                  + 2.*cs0*M1.pk(kh, z_pk)[11]/h**2.)*h**3.) 
    # dark matter redsfhit space quadrupole below
    pk_m2.append((M1.pk(kh, z_pk)[18] 
                  +M1.pk(kh, z_pk)[24]
                  +M1.pk(kh, z_pk)[19] 
                  +M1.pk(kh, z_pk)[25] 
                  +M1.pk(kh, z_pk)[26]  
                  +2.*cs2*M1.pk(kh, z_pk)[12]/h**2.)*h**3.)
    # dark matter redsfhit space hexadecapole below
    pk_m4.append((M1.pk(kh, z_pk)[20] 
                  +M1.pk(kh, z_pk)[27]
                  +M1.pk(kh, z_pk)[28] 
                  +M1.pk(kh, z_pk)[29] 
                  +2.*cs4*M1.pk(kh, z_pk)[13]/h**2.)*h**3.) 
     # galaxy redsfhit space monopole below
    pk_g0.append((M1.pk(kh, z_pk)[15] 
                  +M1.pk(kh, z_pk)[21]
                  + b1*M1.pk(kh, z_pk)[16] 
                  + b1*M1.pk(kh, z_pk)[22] 
                  + b1**2.*M1.pk(kh, z_pk)[17] 
                  + b1**2.*M1.pk(kh, z_pk)[23] 
                  + 0.25*b2**2.*M1.pk(kh, z_pk)[1] 
                  + b1*b2*M1.pk(kh, z_pk)[30]
                  + b2*M1.pk(kh, z_pk)[31] 
                  + b1*bG2*M1.pk(kh, z_pk)[32]
                  + bG2*M1.pk(kh, z_pk)[33]
                  + b2*bG2*M1.pk(kh, z_pk)[4]
                  + bG2**2.*M1.pk(kh, z_pk)[5] 
                  + 2.*cs0*M1.pk(kh, z_pk)[11]/h**2. 
                  + (2.*bG2+0.8*bGamma3)*(b1*M1.pk(kh, z_pk)[7]
                                                   +M1.pk(kh, z_pk)[8]))*h**3.
                  + Pshot 
                  + fz**2.*b4*(kh/h)**2.*(fz**2./9. + 2.*fz*b1/7. 
                                          + b1**2./5)*(35./8.)*M1.pk(kh, z_pk)[13]*h)
    # galaxy redsfhit space quadrupole below
    pk_g2.append((M1.pk(kh, z_pk)[18] 
                  +M1.pk(kh, z_pk)[24]
                  +b1*M1.pk(kh, z_pk)[19] 
                  +b1*M1.pk(kh, z_pk)[25] 
                  +b1**2.*M1.pk(kh, z_pk)[26] 
                  +b1*b2*M1.pk(kh, z_pk)[34]
                  +b2*M1.pk(kh, z_pk)[35] 
                  +b1*bG2*M1.pk(kh, z_pk)[36]
                  +bG2*M1.pk(kh, z_pk)[37]
                  +2.*cs2*M1.pk(kh, z_pk)[12]/h**2. 
                  +(2.*bG2+0.8*bGamma3)*M1.pk(kh, z_pk)[9])*h**3. 
                  +fz**2.*b4*(kh/h)**2.*((fz**2.*70. 
                                      + 165.*fz*b1
                                      +99.*b1**2.)*4./693.)*(35./8.)*M1.pk(kh, z_pk)[13]*h)
    # galaxy redsfhit space hexadecapole below
    pk_g4.append((M1.pk(kh, z_pk)[20] 
                  +M1.pk(kh, z_pk)[27]
                  +b1*M1.pk(kh, z_pk)[28] 
                  +b1**2.*M1.pk(kh, z_pk)[29] 
                  +b2*M1.pk(kh, z_pk)[38] 
                  +bG2*M1.pk(kh, z_pk)[39] 
                  +2.*cs4*M1.pk(kh, z_pk)[13]/h**2.)*h**3.
                  +fz**2.*b4*(kh/h)**2.*((fz**2.*210. 
                                      + 390.*fz*b1
                                      +143.*b1**2.)*8./5005.)*(35./8.)*M1.pk(kh, z_pk)[13]*h)


In [14]:
#############################################
#
# EDE: extract spectra
#
#############################################

hEDE = EDE1.h()
fzEDE = EDE1.scale_independent_growth_factor_f(z_pk)
kvec = np.logspace(-3,np.log10(3),1000) # array of kvec in h/Mpc
twopi = 2.*math.pi
khvecEDE = kvec*hEDE # array of kvec in 1/Mpc


#legarray = []
pk_EDE_lin = []
pk_EDE_loop = []
pk_EDE_tree = []
pk_EDE_full = []
pk_EDE_full_ir = []
pk_EDE_ctr = []
pk_EDE_gg = []
pk_EDE_gm = []
pk_EDE_m0 = []
pk_EDE_m2 = []
pk_EDE_m4 = []
pk_EDE_g0 = []
pk_EDE_g2 = []
pk_EDE_g4 = []
pk_EDE_Id2 = []
pk_EDE_IG2 = []
pk_EDE_Id2d2 = []
pk_EDE_Id2G2 = []
pk_EDE_IG2G2 = []
pk_EDE_FG2 = []

b1EDE = 1.7801562076765094
csEDE = 1. # in units [Mpc/h]^2
b2EDE = -3.3229633099327973
bG2EDE = 0.6643694833607848
bGamma3EDE = 0.
PshotEDE = 6699.631293444604 # in units [Mpc/h]^3
cs0EDE = -22.210780706934052 # in units [Mpc/h]^2
cs2EDE = -45.03173430337345 # in units [Mpc/h]^2
cs4EDE = -5 # in units [Mpc/h]^2
b4EDE = 251.77513281428483# in units [Mpc/h]^4

for kh in khvecEDE:
    pk_EDE_tree.append(EDE.pk(kh,z_pk)[14]*hEDE**3.)
    pk_EDE_loop.append(EDE.pk(kh,z_pk)[0]*hEDE**3.)
    pk_EDE_ctr.append(2*EDE.pk(kh,z_pk)[10]*hEDE)
    # basic real space matter power spectrum without IR resummation below
    pk_EDE_full.append((EDE.pk(kh,z_pk)[0]+EDE.pk(kh,z_pk)[14]+2*cs*EDE.pk(kh,z_pk)[10]/hEDE**2.)*hEDE**3.)
    # linear theory matter power spectrum below
    pk_EDE_lin.append(EDE1.pk_lin(kh,z_pk)*hEDE**3.)
    # real space matter power spectrum below
    pk_EDE_full_ir.append((EDE1.pk(kh,z_pk)[0]+EDE1.pk(kh,z_pk)[14]+2*csEDE*EDE1.pk(kh,z_pk)[10]/hEDE**2.)*hEDE**3.)
    # real space galaxy-galaxy power spectrum below
    pk_EDE_gg.append((b1EDE**2.*EDE1.pk(kh, z_pk)[14] 
                  + b1EDE**2.*EDE1.pk(kh, z_pk)[0] 
                  + 2.*cs0EDE*EDE1.pk(kh, z_pk)[10]/hEDE**2. 
                  + b1EDE*b2EDE*EDE1.pk(kh, z_pk)[2]
                  + 0.25*b2EDE**2.*EDE1.pk(kh, z_pk)[1] 
                  + 2.*b1EDE*bG2EDE*EDE1.pk(kh, z_pk)[3] 
                  + b1EDE*(2.*bG2EDE + 0.8*bGamma3EDE)*EDE1.pk(kh, z_pk)[6] 
                  + bG2EDE**2.*EDE1.pk(kh, z_pk)[5] 
                  + b2EDE*bG2EDE*EDE1.pk(kh, z_pk)[4])*hEDE**3. + PshotEDE) 
    # real space galaxy-matter power spectrum below
    pk_EDE_gm.append((b1EDE*EDE1.pk(kh, z_pk)[14] 
                  + b1EDE*EDE1.pk(kh, z_pk)[0] 
                  + 2.*cs0EDE*EDE1.pk(kh, z_pk)[10]/hEDE**2. 
                  + (b2EDE/2)*EDE1.pk(kh, z_pk)[2]
                  + bG2EDE*EDE1.pk(kh, z_pk)[3])*hEDE**3.) 
    # separate contributions into the galaxy-galaxy power spectrum
    pk_EDE_Id2.append((EDE1.pk(kh, z_pk)[2])*hEDE**3.) 
    pk_EDE_IG2.append((EDE1.pk(kh, z_pk)[3])*hEDE**3.) 
    pk_EDE_Id2d2.append((EDE1.pk(kh, z_pk)[1])*hEDE**3.) 
    pk_EDE_IG2G2.append((EDE1.pk(kh, z_pk)[5])*hEDE**3.) 
    pk_EDE_Id2G2.append((EDE1.pk(kh, z_pk)[4])*hEDE**3.) 
    pk_EDE_FG2.append((EDE1.pk(kh, z_pk)[6])*hEDE**3.) 
    # dark matter redsfhit space monopole below
    pk_EDE_m0.append((EDE1.pk(kh, z_pk)[15] 
                  +EDE1.pk(kh, z_pk)[21]
                  +EDE1.pk(kh, z_pk)[16] 
                  +EDE1.pk(kh, z_pk)[22] 
                  +EDE1.pk(kh, z_pk)[17] 
                  +EDE1.pk(kh, z_pk)[23] 
                  + 2.*cs0EDE*EDE1.pk(kh, z_pk)[11]/hEDE**2.)*hEDE**3.) 
    # dark matter redsfhit space quadrupole below
    pk_EDE_m2.append((EDE1.pk(kh, z_pk)[18] 
                  +EDE1.pk(kh, z_pk)[24]
                  +EDE1.pk(kh, z_pk)[19] 
                  +EDE1.pk(kh, z_pk)[25] 
                  +EDE1.pk(kh, z_pk)[26]  
                  +2.*cs2EDE*EDE1.pk(kh, z_pk)[12]/hEDE**2.)*hEDE**3.)
    # dark matter redsfhit space hexadecapole below
    pk_EDE_m4.append((EDE1.pk(kh, z_pk)[20] 
                  +EDE1.pk(kh, z_pk)[27]
                  +EDE1.pk(kh, z_pk)[28] 
                  +EDE1.pk(kh, z_pk)[29] 
                  +2.*cs4EDE*EDE1.pk(kh, z_pk)[13]/hEDE**2.)*hEDE**3.) 
     # galaxy redsfhit space monopole below
    pk_EDE_g0.append((EDE1.pk(kh, z_pk)[15] 
                  + EDE1.pk(kh, z_pk)[21]
                  + b1EDE*EDE1.pk(kh, z_pk)[16] 
                  + b1EDE*EDE1.pk(kh, z_pk)[22] 
                  + b1EDE**2.*EDE1.pk(kh, z_pk)[17] 
                  + b1EDE**2.*EDE1.pk(kh, z_pk)[23] 
                  + 0.25*b2EDE**2.*EDE1.pk(kh, z_pk)[1] 
                  + b1EDE*b2EDE*EDE1.pk(kh, z_pk)[30]
                  + b2EDE*EDE1.pk(kh, z_pk)[31] 
                  + b1EDE*bG2EDE*EDE1.pk(kh, z_pk)[32]
                  + bG2EDE*EDE1.pk(kh, z_pk)[33]
                  + b2EDE*bG2EDE*EDE1.pk(kh, z_pk)[4]
                  + bG2EDE**2.*EDE1.pk(kh, z_pk)[5] 
                  + 2.*cs0EDE*EDE1.pk(kh, z_pk)[11]/hEDE**2. 
                  + (2.*bG2EDE+0.8*bGamma3EDE)*(b1EDE*EDE1.pk(kh, z_pk)[7]
                                                   +EDE1.pk(kh, z_pk)[8]))*hEDE**3.
                  + PshotEDE 
                  + fzEDE**2.*b4EDE*(kh/hEDE)**2.*(fzEDE**2./9. + 2.*fzEDE*b1EDE/7. 
                                          + b1EDE**2./5)*(35./8.)*EDE1.pk(kh, z_pk)[13]*hEDE)
    # galaxy redsfhit space quadrupole below
    pk_EDE_g2.append((EDE1.pk(kh, z_pk)[18] +EDE1.pk(kh, z_pk)[24]
                  +b1EDE*(EDE1.pk(kh, z_pk)[19] + EDE1.pk(kh, z_pk)[25])
                  +b1EDE**2.*EDE1.pk(kh, z_pk)[26] 
                  +b1EDE*b2EDE*EDE1.pk(kh, z_pk)[34]
                  +b2EDE*EDE1.pk(kh, z_pk)[35] 
                  +b1EDE*bG2EDE*EDE1.pk(kh, z_pk)[36]
                  +bG2EDE*EDE1.pk(kh, z_pk)[37]
                  +2.*cs2EDE*EDE1.pk(kh, z_pk)[12]/hEDE**2. 
                  +(2.*bG2EDE+0.8*bGamma3EDE)*EDE1.pk(kh, z_pk)[9])*hEDE**3. 
                  +fzEDE**2.*b4EDE*(kh/hEDE)**2.*((fzEDE**2.*70. + 165.*fzEDE*b1EDE
                                      +99.*b1EDE**2.)*4./693.)*(35./8.)*EDE1.pk(kh, z_pk)[13]*hEDE)
    
    # galaxy redsfhit space hexadecapole below
    pk_EDE_g4.append((EDE1.pk(kh, z_pk)[20] 
                  +EDE1.pk(kh, z_pk)[27]
                  +b1EDE*EDE1.pk(kh, z_pk)[28] 
                  +b1EDE**2.*EDE1.pk(kh, z_pk)[29] 
                  +b2EDE*EDE1.pk(kh, z_pk)[38] 
                  +bG2EDE*EDE1.pk(kh, z_pk)[39] 
                  +2.*cs4EDE*EDE1.pk(kh, z_pk)[13]/hEDE**2.)*hEDE**3.
                  +fzEDE**2.*b4EDE*(kh/hEDE)**2.*((fzEDE**2.*210. 
                                      + 390.*fzEDE*b1EDE
                                      +143.*b1EDE**2.)*8./5005.)*(35./8.)*EDE1.pk(kh, z_pk)[13]*hEDE)


In [15]:
# Plot

# Create figures
#
fig_Pk, ax_Pk = plt.subplots()
fig_Pkir, ax_Pkir = plt.subplots()
fig_Pkgg, ax_Pkgg = plt.subplots()
fig_Pkgm, ax_Pkgm = plt.subplots()
fig_Pkmz, ax_Pkmz = plt.subplots()
fig_Pkgz, ax_Pkgz = plt.subplots()

# LCDM

ax_Pk.loglog(kvec,np.array(np.abs(pk_loop)),color='purple',linestyle='-',label='1-loop')
ax_Pk.loglog(kvec,np.array(np.abs(pk_ctr)),color='b',linestyle='-',label='Counterterm')
ax_Pk.loglog(kvec,np.array(pk_tree),color='darkorange',linestyle='-',label='Linear')
ax_Pk.loglog(kvec,np.array(pk_full),color='darkgreen',linestyle='-',label='Total')

ax_Pkir.plot(kvec,np.array(pk_lin)*kvec**1.5,color='purple',linestyle='-.',label='linear')
ax_Pkir.plot(kvec,np.array(pk_full)*kvec**1.5,color='b',linestyle='--',label='1-loop, no IR resummation')
ax_Pkir.plot(kvec,np.array(pk_full_ir)*kvec**1.5,color='r',linestyle='-',label='1-loop, IR resummation')

ax_Pkgg.plot(kvec,np.array(pk_gg)*kvec,color='b',linestyle='-',label='galaxy-galaxy')
ax_Pkgg.plot(kvec,np.array(pk_gm)*kvec,color='darkorange',linestyle='-',label='galaxy-matter')
ax_Pkgg.plot(kvec,np.array(pk_full_ir)*kvec,color='purple',linestyle='-',label='matter-matter')

ax_Pkgm.loglog(kvec,np.array(np.abs(pk_lin)),color='k',linestyle='-',label=r'$P_{\rm lin}$')
ax_Pkgm.loglog(kvec,np.array(np.abs(pk_Id2)),color='b',linestyle='-',label=r'${\cal I}_{\delta^2}$')
ax_Pkgm.loglog(kvec,np.array(np.abs(pk_IG2)),color='darkorange',linestyle='--',label=r'${\cal I}_{{\cal G}_2}$')
ax_Pkgm.loglog(kvec,np.array(np.abs(pk_Id2d2)),color='r',linestyle='-.',label=r'${\cal I}_{\delta^2\delta^2}$')
ax_Pkgm.loglog(kvec,np.array(np.abs(pk_Id2G2)),color='teal',linestyle=':',label=r'${\cal I}_{\delta^2{\cal G}_2}$')
ax_Pkgm.loglog(kvec,np.array(np.abs(pk_IG2G2)),color='purple',linestyle='-',label=r'${\cal I}_{{\cal G}_2{\cal G}_2}$')
ax_Pkgm.loglog(kvec,np.array(np.abs(pk_FG2)),color='darkgreen',linestyle='--',label=r'${\cal F}_{{\cal G}_2}$')

ax_Pkmz.plot(kvec,np.array(pk_m0)*kvec,color='darkgreen',linestyle='-',label=r'$\ell = 0$')
ax_Pkmz.plot(kvec,np.array(pk_m2)*kvec,color='darkorange',linestyle='-',label=r'$\ell = 2$')
ax_Pkmz.plot(kvec,np.array(pk_m4)*kvec,color='purple',linestyle='-',label=r'$\ell = 4$')

ax_Pkgz.plot(kvec,np.array(pk_g0)*kvec,color='darkgreen',linestyle='-',label=r'$\ell = 0$')
ax_Pkgz.plot(kvec,np.array(pk_g2)*kvec,color='darkorange',linestyle='-',label=r'$\ell = 2$')
ax_Pkgz.plot(kvec,np.array(pk_g4)*kvec,color='purple',linestyle='-',label=r'$\ell = 4$')

# EDE

ax_Pk.loglog(kvec,np.array(np.abs(pk_EDE_loop)),color='purple',linestyle='-.',label='1-loop EDE')
ax_Pk.loglog(kvec,np.array(np.abs(pk_EDE_ctr)),color='b',linestyle='-.',label='Counterterm  EDE')
ax_Pk.loglog(kvec,np.array(pk_EDE_tree),color='darkorange',linestyle='-.',label='Linear  EDE')
ax_Pk.loglog(kvec,np.array(pk_EDE_full),color='darkgreen',linestyle='-.',label='Total  EDE')

ax_Pkir.plot(kvec,np.array(pk_EDE_lin)*kvec**1.5,color='purple',linestyle='--',label='linear EDE')
ax_Pkir.plot(kvec,np.array(pk_EDE_full)*kvec**1.5,color='b',linestyle='-.',label='1-loop, no IR resummation EDE')
ax_Pkir.plot(kvec,np.array(pk_EDE_full_ir)*kvec**1.5,color='r',linestyle=':',label='1-loop, IR resummation EDE')

ax_Pkgg.plot(kvec,np.array(pk_EDE_gg)*kvec,color='b',linestyle='-.',label='galaxy-galaxy  EDE')
ax_Pkgg.plot(kvec,np.array(pk_EDE_gm)*kvec,color='darkorange',linestyle='-.',label='galaxy-matter EDE')
ax_Pkgg.plot(kvec,np.array(pk_EDE_full_ir)*kvec,color='purple',linestyle='-.',label='matter-matter  EDE')

ax_Pkgm.loglog(kvec,np.array(np.abs(pk_EDE_lin)),color='k',linestyle='-',label=r'$P_{\rm lin}$ EDE')
ax_Pkgm.loglog(kvec,np.array(np.abs(pk_EDE_Id2)),color='b',linestyle='-',label=r'${\cal I}_{\delta^2}$ EDE')
ax_Pkgm.loglog(kvec,np.array(np.abs(pk_EDE_IG2)),color='darkorange',linestyle='--',label=r'${\cal I}_{{\cal G}_2}$ EDE')
ax_Pkgm.loglog(kvec,np.array(np.abs(pk_EDE_Id2d2)),color='r',linestyle='-.',label=r'${\cal I}_{\delta^2\delta^2}$ EDE')
ax_Pkgm.loglog(kvec,np.array(np.abs(pk_EDE_Id2G2)),color='teal',linestyle=':',label=r'${\cal I}_{\delta^2{\cal G}_2}$ EDE')
ax_Pkgm.loglog(kvec,np.array(np.abs(pk_EDE_IG2G2)),color='purple',linestyle='-',label=r'${\cal I}_{{\cal G}_2{\cal G}_2}$ EDE')
ax_Pkgm.loglog(kvec,np.array(np.abs(pk_EDE_FG2)),color='darkgreen',linestyle='--',label=r'${\cal F}_{{\cal G}_2}$ EDE')

ax_Pkmz.plot(kvec,np.array(pk_EDE_m0)*kvec,color='darkgreen',linestyle='-.',label=r'$\ell = 0$ EDE')
ax_Pkmz.plot(kvec,np.array(pk_EDE_m2)*kvec,color='darkorange',linestyle='-.',label=r'$\ell = 2$ EDE')
ax_Pkmz.plot(kvec,np.array(pk_EDE_m4)*kvec,color='purple',linestyle='-.',label=r'$\ell = 4$ EDE')

ax_Pkgz.plot(kvec,np.array(pk_EDE_g0)*kvec,color='darkgreen',linestyle='-.',label=r'$\ell = 0$ EDE')
ax_Pkgz.plot(kvec,np.array(pk_EDE_g2)*kvec,color='darkorange',linestyle='-.',label=r'$\ell = 2$ EDE')
ax_Pkgz.plot(kvec,np.array(pk_EDE_g4)*kvec,color='purple',linestyle='-.',label=r'$\ell = 4 EDE$')


# output of P(k) figures
ax_Pk.set_xlim([1.e-3,1])
ax_Pk.set_ylim([1,5e4])
ax_Pk.set_xlabel(r'$k \,\,\,\, [h\mathrm{Mpc}^{-1}]$')
ax_Pk.set_ylabel(r'$P(k)\,\,\,\, [h^{-1}\mathrm{Mpc}]^3$')
ax_Pk.legend(fontsize='16',ncol=1,loc='upper left')
fig_Pk.savefig('real_Pk.pdf')
fig_Pk.tight_layout()

ax_Pkir.set_xlim([1.e-3,0.5])
ax_Pkir.set_ylim([55,125])
ax_Pkir.set_xlabel(r'$k \,\,\,\, [h\mathrm{Mpc}^{-1}]$')
ax_Pkir.set_ylabel(r'$P(k)k^{3/2}\,\,\,\, [h^{-1}\mathrm{Mpc}]^{3/2}$')
ax_Pkir.legend(fontsize='16',ncol=1,loc='upper right')
fig_Pkir.savefig('real_Pk_IR.pdf')
fig_Pkir.tight_layout()

ax_Pkgg.set_xlim([1.e-3,0.3])
ax_Pkgg.set_ylim([1,2200])
ax_Pkgg.set_xlabel(r'$k \,\,\,\, [h\mathrm{Mpc}^{-1}]$')
ax_Pkgg.set_ylabel(r'$P(k)k\,\,\,\, [h^{-1}\mathrm{Mpc}]^{2}$')
ax_Pkgg.legend(fontsize='16',ncol=1,loc='upper right')
fig_Pkgg.savefig('real_Pkgg.pdf')
fig_Pkgg.tight_layout()

ax_Pkgm.set_xlim([1.e-3,1])
ax_Pkgm.set_ylim([1,5e4])
ax_Pkgm.set_xlabel(r'$k \,\,\,\, [h\mathrm{Mpc}^{-1}]$')
ax_Pkgm.set_ylabel(r'$P(k)\,\,\,\, [h^{-1}\mathrm{Mpc}]^3$')
ax_Pkgm.legend(fontsize='16',ncol=1,loc='upper left')
fig_Pkgm.savefig('real_Pkgg_breakdown.pdf')
fig_Pkgm.tight_layout()

ax_Pkmz.set_xlim([1.e-3,0.3])
ax_Pkmz.set_ylim([1,1000])
ax_Pkmz.set_xlabel(r'$k \,\,\,\, [h\mathrm{Mpc}^{-1}]$')
ax_Pkmz.set_ylabel(r'$P_{\ell,\,mm}(k)k\,\,\,\, [h^{-1}\mathrm{Mpc}]^{2}$')
ax_Pkmz.legend(fontsize='16',ncol=1,loc='upper right')
fig_Pkmz.savefig('rsd_Pkmm.pdf')
fig_Pkmz.tight_layout()


ax_Pkgz.set_xlim([1.e-3,0.3])
ax_Pkgz.set_ylim([1,2500])
ax_Pkgz.set_xlabel(r'$k \,\,\,\, [h\mathrm{Mpc}^{-1}]$')
ax_Pkgz.set_ylabel(r'$P_{\ell,\,gg}(k)k\,\,\,\, [h^{-1}\mathrm{Mpc}]^{2}$')
ax_Pkgz.legend(fontsize='16',ncol=1,loc='upper right')
fig_Pkgz.savefig('rsd_Pkgg.pdf')
fig_Pkgz.tight_layout()


In [16]:
kh = 0.1*hEDE
print(((EDE1.pk(kh, z_pk)[15] 
                  + EDE1.pk(kh, z_pk)[21]
                  + b1EDE*EDE1.pk(kh, z_pk)[16] 
                  + b1EDE*EDE1.pk(kh, z_pk)[22] 
                  + b1EDE**2.*EDE1.pk(kh, z_pk)[17] 
                  + b1EDE**2.*EDE1.pk(kh, z_pk)[23] 
                  + 0.25*b2EDE**2.*EDE1.pk(kh, z_pk)[1] 
                  + b1EDE*b2EDE*EDE1.pk(kh, z_pk)[30]
                  + b2EDE*EDE1.pk(kh, z_pk)[31] 
                  + b1EDE*bG2EDE*EDE1.pk(kh, z_pk)[32]
                  + bG2EDE*EDE1.pk(kh, z_pk)[33]
                  + b2EDE*bG2EDE*EDE1.pk(kh, z_pk)[4]
                  + bG2EDE**2.*EDE1.pk(kh, z_pk)[5] 
                  + 2.*cs0EDE*EDE1.pk(kh, z_pk)[11]/hEDE**2. 
                  + (2.*bG2EDE+0.8*bGamma3EDE)*(b1EDE*EDE1.pk(kh, z_pk)[7]
                                                   +EDE1.pk(kh, z_pk)[8]))*hEDE**3.
                  + PshotEDE 
                  + fzEDE**2.*b4EDE*(kh/hEDE)**2.*(fzEDE**2./9. + 2.*fzEDE*b1EDE/7. 
                                          + b1EDE**2./5)*(35./8.)*EDE1.pk(kh, z_pk)[13]*hEDE))
    # galaxy redsfhit space quadrupole below
print(((EDE1.pk(kh, z_pk)[18] +EDE1.pk(kh, z_pk)[24]
                  +b1EDE*(EDE1.pk(kh, z_pk)[19] + EDE1.pk(kh, z_pk)[25])
                  +b1EDE**2.*EDE1.pk(kh, z_pk)[26] 
                  +b1EDE*b2EDE*EDE1.pk(kh, z_pk)[34]
                  +b2EDE*EDE1.pk(kh, z_pk)[35] 
                  +b1EDE*bG2EDE*EDE1.pk(kh, z_pk)[36]
                  +bG2EDE*EDE1.pk(kh, z_pk)[37]
                  +2.*cs2EDE*EDE1.pk(kh, z_pk)[12]/hEDE**2. 
                  +(2.*bG2EDE+0.8*bGamma3EDE)*EDE1.pk(kh, z_pk)[9])*hEDE**3. 
                  +fzEDE**2.*b4EDE*(kh/hEDE)**2.*((fzEDE**2.*70. + 165.*fzEDE*b1EDE
                    +99.*b1EDE**2.)*4./693.)*(35./8.)*EDE1.pk(kh, z_pk)[13]*hEDE))
    

15335.8980342
5192.10280422
